In [2]:
import plotly.graph_objects as go
import plotly.io
import numpy as np
from dataclasses import dataclass
from datetime import datetime
import os
import math
import FileParser
from IPython.display import clear_output

plotly.io.renderers.default = "chrome"

def normalised_kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    n = len(values1)
    assert len(values2) == n, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(n), np.arange(n))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered / (n * (n - 1))


def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    n = len(values1)
    assert len(values2) == n, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(n), np.arange(n))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered

eps_list = np.linspace(.25, 15, 60)
ratio_list = np.linspace(.25, 15, 60)
neighborhood_size = 5
species = [
    "Ruff", "Groove-billed Ani", "Acorn Woodpecker", "Brown Thrasher", "Eastern Phoebe", "Gray Catbird", "Huttons Vireo", "Lark Bunting", "Lesser Black-backed Gull", "Long-tailed Duck", "Long-tailed Jaeger", "Mew Gull", "Parasitic Jaeger", "Pomarine Jaeger", "Red Phalarope", "Red-faced Warbler", "Sabines Gull"
]

if input("Regnerate Ranks? ") == "y":
    print("Regenerating rank files...")
    FileParser.generate_dbscann_ranked_lists(species, eps_list, ratio_list)

ranking_dict = FileParser.get_ranked_lists_from_file()
matrix = []
eps_checked = []
for coords, ranks in ranking_dict.items():
    ratio_checked = []
    if not coords[0] in eps_checked:
        eps_checked.append(coords[0])
        matrix.append([])
    if not coords[1] in ratio_checked:
        ratio_checked.append(coords[1])
        matrix[eps_checked.index(coords[0])].append(ranks)

matrix = np.array(matrix)
# print(matrix)

in_range_coords = []
diff_matrix = np.zeros((len(matrix), len(matrix[0])))
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        max_diff = 0
        for i1 in range(i-2, i+3, 1):
            if i1 >= 0 and i1 < len(matrix):
                for j1 in range(j-2, j+3, 1):
                    if j1 >= 0 and j1 < len(matrix[i]):
                        if math.dist([eps_list[i], ratio_list[j]], [eps_list[i1], ratio_list[j1]]) < neighborhood_size:
                            diff = kendall_tau_distance(matrix[i][j], matrix[i1][j1])
                            max_diff = diff if diff > max_diff else max_diff
        diff_matrix[i][j] = max_diff

# print(diff_matrix)

def update_grid(trace, points, selector):
    clicked_index = points.point_inds[0]
    clicked_eps = float(points.xs[0])
    clicked_ratio = float(points.ys[0])
    # print(clicked_index)
    # print(clicked_eps)
    # print(clicked_ratio)
    #
    # print(matrix[clicked_index[0]][clicked_index[1]])
    # print(eps_list[clicked_index[1]])
    # print(ratio_list[clicked_index[0]])

    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            diff_matrix[i][j] = kendall_tau_distance(matrix[i][j], matrix[clicked_index[1]][clicked_index[0]])

    # print("EPS={} Ratio={} Ranking:\n".format(eps_list[clicked_index[1]], ratio_list[clicked_index[0]]))
    # print(matrix[clicked_index[1]][clicked_index[0]])
    matrix_text = ""
    rank = 1
    for bird in matrix[clicked_index[1]][clicked_index[0]]:
        matrix_text += str(rank) + ". " + bird + "<br>"
        rank+=1

    with fig.batch_update():
        fig.data[0].z = diff_matrix.T
        fig.update_layout(
            title="Diff with EPS={} Ratio={}".format(eps_list[clicked_index[1]], ratio_list[clicked_index[0]]),
            annotations=[
                go.layout.Annotation(
                    text="EPS={} Ratio={} Ranking:<br>".format(eps_list[clicked_index[1]], ratio_list[clicked_index[0]]) + matrix_text,
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=1.1,
                    y=0.8,
                    bordercolor='black',
                    borderwidth=1
                )
            ]
        )


fig = go.FigureWidget([go.Heatmap(
    x=["%.2f" % number for number in eps_list],
    y=["%.2f" % number for number in ratio_list],
    z=diff_matrix.T,
    colorscale="Blues",
)])

fig.update_layout(
    autosize=True,
    width=800,
    height=800,
    xaxis_title="EPS",
    yaxis_title="Days/EPS",
    title="Max Neighborhood Diff (Neighborhood Radius: " + str(neighborhood_size) + " )",
)

fig.data[0].on_click(update_grid)

fig


Regnerate Ranks? n


FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                             'r…